# Make manual V1, V2 and V3 waveforms

1. Iterate on V1 waveform until happy
    * re-run `ld_V1` overwrites self each time
1. Give `ld_V1` as argument to `ld_V2.draw_line`
    * see in relation to each other
1. Give V1 and V2 to V3

_Hint:_ stabilise baseline by making a couple of points close to the first kink.

In [1]:
%matplotlib qt

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from scipy.misc import imread

Take an image as background. This one is from [@Hagler:2014jd].

In [71]:
imname = 'Hagler2014-Fig8D-cropped.tiff'
img = imread(imname)
fs = 250.  # Hz

In [27]:
class LineDrawer(object):
    def __init__(self, fnum=100, fs=250.,
                 tmin=-0.100, tmax=0.350, color='dodgerblue',
                 bck_img=None):
        self.f_interp = None
        
        self.color = color
        self.bck_img = bck_img
        self.time = np.arange(tmin, tmax, 1./fs)
        self.fig, self.ax = plt.subplots(1, 1, num=fnum)
        self.plot_handle = self.ax.plot([self.time[0],
                                         self.time[-1]],
                                        [0, 0])
        self.ax.set_ylim([-1, 1])
        self.ax.grid(True)

        if self.bck_img is not None:
            self.ax.imshow(self.bck_img, zorder=0,
                           extent=[-0.1, 0.35,
                                   -1., 1.], aspect='auto')
    @property
    def amplitude(self):
        return(self.f_interp(self.time))
    
    def plot_line(self, x, y, color='k', linewidth=2.0):
        self.ax.plot(x, y, color=color, linewidth=linewidth)
        self.ax.figure.canvas.draw()
        
    def draw_line(self, V1=None, V2=None, V3=None):
        
        if V1 is not None:
            self.plot_line(V1.time, V1.amplitude,
                           color=V1.color, linewidth=3.0)
        if V2 is not None:
            self.plot_line(V2.time, V2.amplitude,
                           color=V2.color, linewidth=3.0)

        if V3 is not None:
            self.plot_line(V3.time, V3.amplitude,
                           color=V3.color, linewidth=3.0)
        
        xy = self.fig.ginput(n=-1, timeout=300.)
#         xy.insert(0, (self.time[0], 0.))
#         xy.insert(1, (self.time[0] / 2., 0.))
#         xy.insert(2, (0., 0.))
#         xy.append((self.time[-1], 0.))

        x = [p[0] for p in xy]
        y = [p[1] for p in xy]

        self.xy = [x, y]
        
        self.f_interp = UnivariateSpline(x, y, s=0)

        self.plot_line(self.time, self.amplitude,
                       color=self.color, linewidth=3.0)
        


In [28]:
#plt.plot([1,2,3,4,5])
ld_V1 = LineDrawer(fnum=1, color='dodgerblue', bck_img=img)

In [29]:
ld_V1.draw_line() # here you click on the plot

/Users/cjb/miniconda_envs/oldqt/lib/python3.5/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [39]:
ld_V2 = LineDrawer(fnum=2, color='lime', bck_img=img)

In [40]:
ld_V2.draw_line(V1=ld_V1)

/Users/cjb/miniconda_envs/oldqt/lib/python3.5/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [37]:
ld_V3 = LineDrawer(fnum=3, color='crimson', bck_img=img)

In [38]:
ld_V3.draw_line(V1=ld_V1, V2=ld_V2)

/Users/cjb/miniconda_envs/oldqt/lib/python3.5/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


## Save numpy array

In [59]:
waveforms = np.empty((len(ld_V1.time), 4))
waveforms[:, 0] = ld_V1.time
waveforms[:, 1] = ld_V1.amplitude
waveforms[:, 2] = ld_V2.amplitude
waveforms[:, 3] = ld_V3.amplitude

Find baseline samples, mean over time, then mean over 3 regions

In [60]:
bl = np.mean(np.mean(waveforms[waveforms[:, 0] < 0., :], axis=0)[1:])
waveforms[:, 1:] -= bl

Scale V1 to value given in Table 1 (95% contrast)

In [61]:
peak = np.max(np.abs(waveforms[:, 1:]))
waveforms[:, 1:] *= 15.9 / peak

In [68]:
# fig, ax = plt.subplots(1,1)
# ax.plot(waveforms[:, 0], waveforms[:, 1:])

In [72]:
np.save(imname.split('.')[0] + '.npy', waveforms)